In [1]:
#required libraries
%matplotlib qt
import numpy as np
import hyperspy.api as hp
from ncempy.io import dm
import matplotlib.pyplot as plt

In [2]:
#load spectrum image
dataset = dm.dmReader("D:\Documents\Summer 2020 Research\Sample Data\SI_2.dm4")
psf = dm.dmReader("D:\Documents\Summer 2020 Research\Sample Data\psf.dm4")

#index for separate spectra of the same size
Spec = dataset['data'][:1778,0,0]
PSF = psf['data'][:,0,0]

dataset.keys()

dict_keys(['filename', 'data', 'pixelUnit', 'pixelSize', 'pixelOrigin', 'coords'])

In [51]:
#Functions

#Normalize spectrum based on total electron counts
def normalizeS(S):
	S_norm = S/np.sum(S)
	return S_norm

#ISRA algorithm
def ISRA(iterations, PSF_norm, Spec):
    ISRA5 = np.copy(Spec)
    ISRA1 = np.convolve(PSF_norm, Spec, mode='same')
    for i in range(iterations):
        ISRA2 = np.convolve(PSF_norm, ISRA5, mode='same')
        ISRA3 = np.convolve(PSF_norm, ISRA2, mode='same')
        ISRA4 = np.divide(ISRA1, ISRA3)
        ISRA5 = np.multiply(ISRA4, ISRA5)
    return ISRA5

#Richardson-Lucy algorithm (from Edson Bellido)
def RL(iterations, PSF_norm, Spec):
    RL4 = np.copy(Spec)
    for ii in range(iterations):
        RL1 = np.convolve(PSF_norm, RL4, mode='same')
        RL2 = np.divide(Spec,RL1)
        RL3 = np.convolve(PSF_norm, RL2, mode='same')
        RL4 = np.multiply(RL3, RL4)
    return RL4

In [48]:
#normalize the PSF
PSF_norm = normalizeS(PSF)
Spec_norm = normalizeS(Spec)

In [49]:
#perform deconvolution
ISRA = ISRA(20, PSF_norm, Spec_norm)
RL = RL(20, PSF_norm, Spec_norm)

In [50]:
#plot

fig = plt.figure()
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)
ax1.set_title("Spec")
ax2.set_title("PSF")
ax1.plot(Spec_norm)
ax2.plot(PSF_norm)

fig_1 = plt.figure()
ax_test1 = fig_1.add_subplot(111)
ax_test1.plot(abs(ISRA))
fig_1.suptitle("ISRA")
ax_test1.set_ylim(0,0.02)

fig_2 = plt.figure()
ax_test2 = fig_2.add_subplot(111)
ax_test2.plot(abs(RL))
fig_2.suptitle("RL")
ax_test2.set_ylim(0,0.02)

plt.show()